# Introduction
What is the objective of this notebook? </br>
The objective of this notebook is to create a scipt which can make a repository of recipes and their ingredients. </br>

Below i will import the necessary libraries

In [1]:
import pandas as pd
import os
from datetime import datetime
from tqdm import tqdm
import regex as re
import time
import numpy as np
import random
import pyperclip as py
import openai 
import os
import pandas as pd
# from    dotenv import load_dotenv
# load_dotenv()
# openai.api_type     = os.getenv('OPENAI_TYPE')
# openai.api_base     = os.getenv('OPENAI_BASE')
# openai.api_version  = os.getenv('OPENAI_VERSION')
# openai.api_key      = os.getenv('OPENAI_KEY')

# import utilities as my

In [2]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
from tqdm import tqdm

def urls_on_same_domain(url, max_urls=5000):
    """
    Counts the number of unique URLs on the same domain as the given URL,
    iterating through links until no more are found or a max limit is reached.
    The progress is displayed using tqdm.

    Args:
    url (str): The starting URL to fetch and analyze.
    max_urls (int): Maximum number of URLs to fetch.

    Returns:
    int: The total number of unique URLs found on the same domain.
    """
    visited_urls = set()
    urls_to_visit = set([url])
    domain = urlparse(url).netloc

    with tqdm(total=max_urls, desc=f"Analyzing URLs currently found {len(visited_urls)}, and have {len(urls_to_visit)} to go", unit="url") as pbar:
        while len(urls_to_visit) != 0 and len(urls_to_visit) < max_urls:
            current_url = urls_to_visit.pop()
            try:
                response = requests.get(current_url)
                response.raise_for_status()

                soup = BeautifulSoup(response.content, 'html.parser')
                anchors = soup.find_all('a', href=True)

                for anchor in anchors:
                    href = anchor['href']
                    full_url = urljoin(current_url, href)
                    href_domain = urlparse(full_url).netloc

                    if href_domain == domain and full_url not in visited_urls and '#comment' not in full_url and '#respond' not in full_url:
                        urls_to_visit.add(full_url)

                visited_urls.add(current_url)
                pbar.update(1)
                pbar.set_description(f"Analyzing URLs currently found {len(visited_urls)}, and have {len(urls_to_visit)} to go")
            except requests.RequestException:
                # Handle exceptions for requests
                continue

    return visited_urls, urls_to_visit

In [3]:
import requests
from bs4 import BeautifulSoup

def get_text_from_url(url):
    """
    Fetches the content from a given URL and returns the text separated by lines.
    If the URL gets redirected more than twice, returns 'redirect link'.

    Args:
    url (str): The URL from which to fetch the content.

    Returns:
    str: A string containing the text from the URL, separated by lines, or 'redirect link'.
    """
    # Send a GET request to the URL with redirection allowed
    response = requests.get(url, allow_redirects=True)
    
    # Check the number of redirects
    if len(response.history) > 2:
        return 'redirect link'
    
    # Raise an exception if the request was unsuccessful
    response.raise_for_status()

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract and return the text, separated by lines
    return '\n'.join(soup.stripped_strings)


In [4]:
recipes = pd.read_csv('recipes_3.csv')
ingredients_df = pd.read_csv('ingredient_df.csv')
pantry = pd.read_csv('pantry_2.csv')
# lets go to the download folder and import the ingredients_3.csv file
conversion = pd.read_csv('conversion.csv')


In [5]:
recipes

,urls,text,fullfillment,dinner,addon,vegeterian,price_add,matches
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,Indisk vegetarbowl med rispilaf og sprød blomk...,0.0,1,0,1,7714.0,0.434783
1,https://www.valdemarsro.dk/pasta-med-graeskar/,Pasta med græskar og masser af god smag - opsk...,0.0,1,0,1,6415.0,0.363636
2,https://www.valdemarsro.dk/blomkaal-og-halloum...,Blomkål og halloumi i sursød sauce med dejlig ...,0.0,1,0,1,6278.0,0.250000
3,https://www.valdemarsro.dk/graeskarmuffins/,Græskarmuffins - opskrift på saftige muffins m...,0.0,0,0,0,5100.0,0.200000
4,https://www.valdemarsro.dk/vegetar-taco/,Vegetar taco med krydret bagt blomkål og mango...,0.0,1,0,1,4967.0,0.250000
...,...,...,...,...,...,...,...,...
3255,Kikærte og mango curry med basmati ris,Navn: Kikærte og mango curry med basmati ris\n...,0.0,0,0,0,0.0,0.000000
3256,Kikærte- og kyllingefad med avocado og cherryt...,Navn: Kikærte- og kyllingefad med avocado og c...,0.0,0,0,0,0.0,0.000000
3257,Persille-ris med kikærter og yoghurt-dressing,Navn: Persille-ris med kikærter og yoghurt-dre...,0.0,0,0,0,0.0,0.000000
3258,Avocado og kikærte tacos,Navn: Avocado og kikærte tacos\n\nIngredienser...,0.0,0,0,0,0.0,0.000000


In [6]:
pantry

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price,pareto,in_recipe,matches
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.0,spsk,mayonnaise,g,0.0,40.0,350.0,40.0,True,0,0.173913
1,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.0,stk,æg,stk,0.0,36.0,10.0,36.0,True,1,0.173913
2,https://www.valdemarsro.dk/indisk-vegetar-bowl/,0.5,tsk,røget paprika,stk,1.0,0.0,1.0,0.0,True,0,0.173913
3,https://www.valdemarsro.dk/indisk-vegetar-bowl/,250.0,g,blomkål,stk,0.0,30.0,1.0,30.0,True,0,0.173913
4,https://www.valdemarsro.dk/indisk-vegetar-bowl/,50.0,g,pankorasp,NaN,NaN,NaN,NaN,0.0,False,0,0.173913
...,...,...,...,...,...,...,...,...,...,...,...,...
27052,Kikærte og avocado salat med persille og tomat,50,g,bredbladet persille,NaN,0.0,0.0,0.0,0.0,0,0,0.000000
27053,Kikærte og avocado salat med persille og tomat,1/2,stk,rød chili,NaN,0.0,0.0,0.0,0.0,0,0,0.000000
27054,Kikærte og avocado salat med persille og tomat,2,fed,hvidløg,NaN,0.0,0.0,0.0,0.0,0,0,0.000000
27055,Kikærte og avocado salat med persille og tomat,1/2,stk,citron,NaN,0.0,0.0,0.0,0.0,0,0,0.000000


In [7]:
not_numbers = []

for i in pantry.amounts.unique():
    try:
        float(i)
    except:
        not_numbers.append(i)

In [11]:
not_numbers

['1/2', '1/4', 'Saft', '8-10', 'Saften']

In [12]:
pantry[pantry.amounts == 'Saft']

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price,pareto,in_recipe,matches
22315,Krydret kartoffelris med blomkål og koriander,Saft,stk,af 1 lime,NaN,0.0,0.0,0.0,0.0,0,0,0.0
22419,Græskar risotto med persille og grønkål,Saft,stk,og skal af 1 citron,NaN,0.0,0.0,0.0,0.0,0,0,0.0
23422,Kartoffel- og ingefærmos med krydret agurksalat,Saft,stk,af 1 lime,NaN,0.0,0.0,0.0,0.0,0,0,0.0
23591,Kikærte-avocado salat med tahin dressing,Saft,stk,fra 1 citron,NaN,0.0,0.0,0.0,0.0,0,0,0.0
23893,Avocado risotto med spicy sriracha sauce,Saft,stk,fra 2 lime,NaN,0.0,0.0,0.0,0.0,0,0,0.0
23914,Avocado og grøntsagsris med spicy sriracha sauce,Saft,stk,af 2 lime,NaN,0.0,0.0,0.0,0.0,0,0,0.0
24085,Italiensk inspireret avocado-pasta,Saft,stk,af 2 lime,NaN,0.0,0.0,0.0,0.0,0,0,0.0
24123,Aubergine og champignon gryde med persille og ...,Saft,stk,fra 2 lime,NaN,0.0,0.0,0.0,0.0,0,0,0.0
24677,Kartoffel- og gulerodstærte med persillepesto,Saft,stk,fra 1/2 citron,NaN,0.0,0.0,0.0,0.0,0,0,0.0
25023,Kartoffel- og broccoli-frittata med persille- ...,Saft,stk,af 2 lime,NaN,0.0,0.0,0.0,0.0,0,0,0.0


In [45]:
# pantry.loc[pantry.ingredients == 'nan grøntsagsbouillon', 'amounts'] = 50
# pantry.loc[pantry.ingredients == cleaner.at[0, 'ingredients'], 'units'] = 'g'
# pantry.loc[pantry.ingredients == cleaner.at[0, 'ingredients'], 'ingredients'] = 'pasta'
pantry.loc[pantry.ingredients == 'cremefraiche 8 %', 'ingredients'] = 'cremefraiche'
pantry.loc[pantry.ingredients == 'cremefraiche 18 %', 'ingredients'] = 'cremefraiche'


In [41]:
pantry[pantry.ingredients == 'gram pasta']

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price,pareto,in_recipe,matches


In [61]:
all_ingredients = ingredients_df.ingredients.unique()
cleaner = pantry[~pantry.ingredients.isin(all_ingredients)].ingredients.value_counts().reset_index()
cleaner.columns = ['ingredients', 'count']

# now lets make an column where we replace the tsk or spsk with nothing
# for ing in cleaner[cleaner.has_measurements == True].ingredients:
#     x = ing.replace('tsk', '')
#     x = x.replace('spsk', '')
#     x = x.replace('  ', '')
#     x = x.strip()
#     cleaner.loc[cleaner.ingredients == ing, 'rep_ingredients'] = x

#     y = cleaner[cleaner.ingredients == ing].has_measurements.values[0]
#     if y == True:
#         pantry.loc[pantry.ingredients == ing, 'ingredients'] = x

# cleaner

In [65]:
cleaner.to_csv('cleaner.csv', index=False)

In [59]:
        pantry.loc[pantry.ingredients == ing, 'ingredients'] = x

In [43]:
cleaner.at[0, 'ingredients']

'cremefraiche 8 %'

In [64]:
for ing in all_ingredients:
    if 'gulerod' in ing:
        print(ing)

gulerod


In [31]:
pantry[pantry.ingredients == 'flagesalt']

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price,pareto,in_recipe,matches
7,https://www.valdemarsro.dk/indisk-vegetar-bowl/,0.5,tsk,flagesalt,NaN,NaN,NaN,NaN,0.0,NaN,0,0.173913
246,https://www.valdemarsro.dk/salt-karamel-is/,0.25,tsk,flagesalt,NaN,NaN,NaN,NaN,0.0,NaN,0,0.000000
609,https://www.valdemarsro.dk/aeggemuffins-opskrift/,0.5,tsk,flagesalt,NaN,NaN,NaN,NaN,0.0,NaN,0,0.333333
745,https://www.valdemarsro.dk/squash-frikadeller/,0.5,tsk,flagesalt,NaN,NaN,NaN,NaN,0.0,NaN,0,0.222222
1081,https://www.valdemarsro.dk/kylling-med-svampes...,0.5,tsk,flagesalt,NaN,NaN,NaN,NaN,0.0,NaN,0,0.117647
...,...,...,...,...,...,...,...,...,...,...,...,...
16564,https://www.valdemarsro.dk/ribbensteg-med-spro...,1.0,tsk,flagesalt,NaN,NaN,NaN,NaN,0.0,NaN,0,0.000000
16610,http://www.valdemarsro.dk/karamelbrud-med-nodder/,0.5,tsk,flagesalt,NaN,NaN,NaN,NaN,0.0,NaN,0,0.142857
16709,http://www.valdemarsro.dk/confit-de-canard,0.5,spsk,flagesalt,NaN,NaN,NaN,NaN,0.0,NaN,0,0.000000
16805,http://www.valdemarsro.dk/kryddersmoer-2/,0.5,tsk,flagesalt,NaN,NaN,NaN,NaN,0.0,NaN,0,0.100000


In [30]:
cleaner

,index,ingredients
0,flagesalt,173
1,gram pasta,141
2,cremefraiche 8 %,78
3,tsk stødt spidskommen,75
4,tsk stødt koriander,66
...,...,...
284,pizzadej (tilføjelse),1
285,spsk sesamfrø,1
286,spsk sesamolie,1
287,spsk soja,1


In [25]:
pantry[pantry.ingredients == 'nan grøntsagsbouillon']

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price,pareto,in_recipe,matches
17038,One-Pot Pasta med Spinat og Soltørrede Tomater,50,g,nan grøntsagsbouillon,NaN,0.0,0.0,0.0,0.0,0,0,0.0
17102,One-Pot Pasta med Spinat og Soltørrede Tomater,50,g,nan grøntsagsbouillon,NaN,0.0,0.0,0.0,0.0,0,0,0.0
17123,One-Pot Pasta med Soltørrede Tomater og Spinat,50,g,nan grøntsagsbouillon,NaN,0.0,0.0,0.0,0.0,0,0,0.0
17219,One-Pot Pasta med Spinat og Soltørrede Tomater,50,g,nan grøntsagsbouillon,NaN,0.0,0.0,0.0,0.0,0,0,0.0
17240,One-Pot Pasta med Spinat og Soltørrede Tomater,50,g,nan grøntsagsbouillon,NaN,0.0,0.0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
25917,Blomkål og kikærte curry med persilleolie og p...,50,g,nan grøntsagsbouillon,NaN,0.0,0.0,0.0,0.0,0,0,0.0
25982,Persille- og kikærteris med røget paprika,50,g,nan grøntsagsbouillon,NaN,0.0,0.0,0.0,0.0,0,0,0.0
26290,Spaghetti med hasselnøddepesto og spinat,50,g,nan grøntsagsbouillon,NaN,0.0,0.0,0.0,0.0,0,0,0.0
26341,Spaghetti med spinat og parmesan,50,g,nan grøntsagsbouillon,NaN,0.0,0.0,0.0,0.0,0,0,0.0


In [20]:
cleaner

,index,ingredients
0,nan grøntsagsbouillon,354
1,flagesalt,173
2,gram pasta,141
3,cremefraiche 8 %,78
4,tsk stødt spidskommen,75
...,...,...
285,pizzadej (tilføjelse),1
286,spsk sesamfrø,1
287,spsk sesamolie,1
288,spsk soja,1


In [14]:
matching = pd.DataFrame(columns=['urls','amounts', 'measurements', 'ingredients'])

new_df = pd.read_csv('conversion.csv')
measurements = new_df['units'].unique()
measurements

def ingredients_cleaner(text):
    try:
        text = text.split('Ingredienser')[1]
        text = text.split('Udskriv')[0]
        text = text.strip()
    except:
        text = ''
    if 'Archive' in text:
        text = ''
    return text

def ingredients_setter(ingredients_text, url):
    ingredients_text = ingredients_cleaner(ingredients_text)    

    for line in ingredients_text.splitlines():
    # now lets see if the first item is a number or not
        pattern = '^\d+'
    # lets utelise the starts with function here, and combine it with some regex
        match = re.findall(pattern, line)
        local_measurement = ''
        if len(match) > 0:
            # if we get a match, we know that the first item is a number
            # so lets split the line at the first space
            amount = line.split(' ', 1)[0]
            # and then print the first item
            # now lets see if there is any measurements in the line
            for measurement in measurements:
                if f' {measurement} ' in line:
                # if there is, lets split the line at the measurement
                    local_measurement = measurement
                # and then print the first item
                    break
            else:
            # if there is no measurement, just print the first item
                local_measurement = 'stk'

            if ',' in line:
                new_line = line.split(', ')[0]
            else:
                new_line = line

            if '.' in new_line:
                new_line = line.split('. ')[0]
            else:
                new_line = new_line

            new_line = new_line.replace(amount, '')
            new_line = new_line.replace(f' {local_measurement} ', '')
            new_line = new_line.strip()
        
            if ',' in amount:
                amount = amount.replace(',', '.')

            matching.loc[len(matching)] = [url, amount, local_measurement, new_line]

test_url = df[df.urls.str.contains('https://www.valdemarsro.dk/hjemmebagte-boller')].urls.values[0]
test_text = df[df.urls.str.contains('https://www.valdemarsro.dk/hjemmebagte-boller')].text.values[0]

# ingredients_setter(test_text, test_url)

matching

,urls,amounts,measurements,ingredients


In [15]:
for i in tqdm(range(len(df))):
    row = df.iloc[i]
    ingredients_setter(row.text, row.urls)

  0%|          | 0/3997 [00:00<?, ?it/s]

100%|██████████| 3997/3997 [01:01<00:00, 64.86it/s] 


In [16]:
matching.amounts = matching.amounts.astype(float)
matching.amounts = matching.amounts / 2

In [17]:
ingredients = pd.read_csv('ingredients.csv')
ingredients.drop_duplicates(subset=['ingredients', 'unit'], inplace=True)
ingredients.reset_index(inplace=True, drop=True)
ingredients.rename(columns={'unit':'purchase_unit'}, inplace=True)

In [18]:
mergeable = ingredients[['ingredients', 'purchase_unit', 'spice', 'price', 'purchase_size']]
matching = matching.merge(mergeable, how='left', on='ingredients')

In [19]:
matching.rename(columns={'measurements':'units'}, inplace=True)

In [20]:
# now in the matching dataframe lets add a column for the price of the ingredient in that dish
conversion = pd.read_csv('conversion.csv')

def set_individual_price(row):
    amount = row['amounts']
    purchase_size = row['purchase_size']
    price = row['price']
    input_units = row['units']
    purchase_unit = row['purchase_unit']
    if len(conversion[(conversion.units == input_units) & (conversion.purchase_unit == purchase_unit)] ) > 0:
        faktor = conversion[(conversion.units == input_units) & (conversion.purchase_unit == purchase_unit)].rate.values[0]
    else:
        return 0

    if purchase_size == 0:
        return 0
    else:
        if input_units == purchase_unit:
            y = np.ceil(amount / purchase_size) * price if amount > purchase_size else price
        else:
            if input_units == 'g' and purchase_unit == 'stk':
                y = price
            else:
                # we need to make it so that it looks in the conversion table and then returns the 
                y = np.ceil(amount * faktor / purchase_size) * price  if amount > purchase_size else price
    
    # lets also round the product of faktor by purchase_size and amount
    z = np.ceil(amount / purchase_size) * faktor if amount > purchase_size else faktor * amount
    
    return y

matching['local_price'] = matching.apply(set_individual_price, axis=1)

In [21]:
matching

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.0,spsk,mayonnaise,g,0.0,40.0,350.0,40.0
1,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.0,stk,æg,stk,0.0,36.0,10.0,36.0
2,https://www.valdemarsro.dk/indisk-vegetar-bowl/,0.5,tsk,røget paprika,stk,1.0,0.0,1.0,0.0
3,https://www.valdemarsro.dk/indisk-vegetar-bowl/,250.0,g,blomkål,stk,0.0,30.0,1.0,30.0
4,https://www.valdemarsro.dk/indisk-vegetar-bowl/,50.0,g,pankorasp,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...
27593,https://www.valdemarsro.dk/nem-pastaret-med-la...,0.5,stk,løg,stk,0.0,15.0,6.0,15.0
27594,https://www.valdemarsro.dk/nem-pastaret-med-la...,1.0,fed,hvidløg,fed,0.0,30.0,12.0,30.0
27595,https://www.valdemarsro.dk/nem-pastaret-med-la...,0.5,dl,piskefløde,ml,0.0,500.0,27.0,500.0
27596,https://www.valdemarsro.dk/nem-pastaret-med-la...,0.5,håndfuld,frisk basilikum,stk,1.0,0.0,1.0,0.0


In [22]:
ingredients = ingredients[ingredients.spice.notna()].copy()
# now lets remove all the items im not using, and then insert the items from the grouped view

In [23]:
ingredients.reset_index(inplace=True, drop=True)

In [24]:
grouped_view = matching.groupby('ingredients').agg({'urls':'count'})
grouped_view.sort_values(by='urls', inplace=True, ascending=False)
grouped_view.reset_index(inplace=True)

In [25]:
grouped_view

,ingredients,urls
0,smør,1071
1,olivenolie,1024
2,sukker,1015
3,hvidløg,860
4,æg,794
...,...,...
859,kogevand fra pastaen,1
860,svinekrone,1
861,durum mel,1
862,dåser flåede tomater,1


In [26]:
for i in tqdm(range(len(grouped_view))):
    ingredient = grouped_view.at[i, 'ingredients']
    urls = grouped_view.at[i, 'urls']

    if ingredient in ingredients.ingredients.values:
        ingredients.loc[ingredients.ingredients == ingredient, 'urls'] = urls
    else:
        ingredients.loc[len(ingredients)] = [ingredient,urls ,'new', '', '', '', '', '']

  0%|          | 0/864 [00:00<?, ?it/s]

100%|██████████| 864/864 [00:01<00:00, 515.55it/s] 


In [27]:
ingredients.sort_values(by='urls', inplace=True, ascending=False)

In [28]:
ingredients

,ingredients,urls,pareto,purchase_unit,spice,faktor,price,purchase_size
0,smør,1071,True,g,0.0,1.0,12.0,5.0
1,olivenolie,1024,True,ml,0.0,15.0,160.0,1000.0
2,sukker,1015,True,g,0.0,1.0,25.0,1000.0
3,hvidløg,860,True,fed,0.0,1.0,30.0,12.0
4,æg,794,True,stk,0.0,1.0,36.0,10.0
...,...,...,...,...,...,...,...,...
805,mandelolie,1,new,,,,,
804,bearnaise,1,new,,,,,
803,padron,1,new,,,,,
802,rapsolie,1,new,,,,,


In [29]:
matching.to_csv('matching.csv', index=False)

In [38]:
df

,urls,text,fullfillment,dinner,addon,vegeterian,price_add,matches
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,Indisk vegetarbowl med rispilaf og sprød blomk...,0.833333,1.0,0.0,0.0,7806.0,3.0
1,https://www.valdemarsro.dk/blomkaalsgratin/,Blomkålsgratin - her får du en skøn opskrift p...,1.000000,1.0,0.0,0.0,7709.0,0.0
2,https://www.valdemarsro.dk/pasta-med-graeskar/,Pasta med græskar og masser af god smag - opsk...,1.000000,1.0,0.0,1.0,6471.0,2.0
3,https://www.valdemarsro.dk/blomkaal-og-halloum...,Blomkål og halloumi i sursød sauce med dejlig ...,1.000000,1.0,0.0,1.0,6313.0,3.0
4,https://www.valdemarsro.dk/groentsagsgryde-med...,"Grøntsagsgryde med kikærter - nem, varmende og...",1.000000,1.0,0.0,1.0,5203.0,2.0
...,...,...,...,...,...,...,...,...
3992,https://www.valdemarsro.dk/buddha-bowl/,Buddha Bowl - få her en nem og lækker opskrift...,NaN,NaN,NaN,NaN,NaN,NaN
3993,https://www.valdemarsro.dk/sommerhyggeaften/,Sommerhyggeaften - Valdemarsro\nOpskrifter\nOp...,NaN,NaN,NaN,NaN,NaN,NaN
3994,https://www.valdemarsro.dk/braiseret-svinenakke/,braiseret svinenakke - opskrift på lækker dans...,NaN,NaN,NaN,NaN,NaN,NaN
3995,https://www.valdemarsro.dk/sma-glimt-kreta/,Charterferie på Kreta - om vores dejlige oplev...,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# now lets make a function which takes a string and looks for the category in a segment of the text
test_text = df.iloc[0].text
test_url  = df.iloc[0].urls

In [51]:
def find_dinner(test_text, test_url):
    segment = test_text.split(test_url)[1]
    if 'Aftensmad' in segment:
        if 'Tilbehør Aftensmad' in segment:
            return False
        return True
    else:
        return False
def find_vegeterian(test_text, test_url):
    segment = test_text.split(test_url)[1]
    if 'Vegetar' in segment:
        return True
    else:
        return False
def find_addon(test_text, test_url):
    segment = test_text.split(test_url)[1]
    if 'Tilbehør Aftensmad' in segment:
        return True
    else:
        return False

for i in range(len(df)):
    row = df.iloc[i]
    try:
        df.loc[i, 'dinner'] = find_dinner(row.text, row.urls)
        df.loc[i, 'vegeterian'] = find_vegeterian(row.text, row.urls)
        df.loc[i, 'addon'] = find_addon(row.text, row.urls)
    except:
        df.loc[i, 'dinner'] = False
        df.loc[i, 'vegeterian'] = False
        df.loc[i, 'addon'] = False


In [52]:
df[(df.dinner == True) & (df.vegeterian == True)]

,urls,text,fullfillment,dinner,addon,vegeterian,price_add,matches
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,Indisk vegetarbowl med rispilaf og sprød blomk...,0.833333,True,False,True,7806.0,3.0
1,https://www.valdemarsro.dk/blomkaalsgratin/,Blomkålsgratin - her får du en skøn opskrift p...,1.000000,True,False,True,7709.0,0.0
2,https://www.valdemarsro.dk/pasta-med-graeskar/,Pasta med græskar og masser af god smag - opsk...,1.000000,True,False,True,6471.0,2.0
3,https://www.valdemarsro.dk/blomkaal-og-halloum...,Blomkål og halloumi i sursød sauce med dejlig ...,1.000000,True,False,True,6313.0,3.0
4,https://www.valdemarsro.dk/groentsagsgryde-med...,"Grøntsagsgryde med kikærter - nem, varmende og...",1.000000,True,False,True,5203.0,2.0
...,...,...,...,...,...,...,...,...
3905,https://www.valdemarsro.dk/mexicansk-risret/,Mexicansk risret - skøn opskrift på en lækker ...,NaN,True,False,True,NaN,NaN
3929,https://www.valdemarsro.dk/falafelvaffel/,Falafelvaffel med lækker topping - opskrift på...,NaN,True,False,True,NaN,NaN
3963,https://www.valdemarsro.dk/falafler/,Falafler - Opskrift på den bedste fabelagtig s...,NaN,True,False,True,NaN,NaN
3969,https://www.valdemarsro.dk/speltotto-med-aerte...,Speltotto med friske ærter og citron – en lækk...,NaN,True,False,True,NaN,NaN


In [53]:
df.to_csv('recipes_2.csv', index=False)

In [4]:
with open("file.txt", "w") as output:
    for url in urls:
        output.write(str(url) + '\n')

In [3]:
with open("file_2.txt", "w") as output:
    for url in to_visit:
        output.write(str(url) + '\n')